## Redundanz



Bestimmen Sie die Entropie eines selbstausgewählten Musiksignals. Führen Sie die Differenzcodierung durch und bestimmen Sie die Entropie erneut!

init.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile

Musik-Datei einlesen:



In [1]:
S_filename = "/home/jbatke/Version/CMD_MMI/Vorlesung/1Battle_Royal.wav"
[f_rate, x_data] = wavfile.read(S_filename)
x_L = np.array(x_data, dtype=float)
plt.plot(x_L)

/tmp/ipykernel_1665085/2883443156.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  [f_rate, x_data] = wavfile.read(S_filename)
| <matplotlib.lines.Line2D | at | 0x7f1dd912d3d0> | <matplotlib.lines.Line2D | at | 0x7f1dd9124910> |
[[./.ob-jupyter/9c7a81d5a4afea5daa9bc971328368c6cdef1d7e.png]]

WDF bestimmen: Schätzung der Wahrscheinlichkeitsdichtefunktion $f_{\vec x}(x)$ per Häufigkeitsdichte



In [1]:
# x_data = np.random.lognormal(0, 1, 10000)
bins_x = np.arange(np.min(np.min(x_data)), np.max(np.max(x_data))+1)
f_1_x, bin_edge = np.histogram(x_data[:,0], bins_x, density=True)
plt.plot(bins_x[1:bins_x.size], f_1_x*100)
plt.xlabel("$x$")
plt.ylabel("rel. Haeufigkeit in %")

Text(0, 0.5, 'rel. Haeufigkeit in %')
[[./.ob-jupyter/a1ba02fcb8d48bf3f53d2715076e24ecc5cb9c01.png]]

Entropie $H(x) = \sum_{i} P_{\vec x}(x_{i}) \log_{2}(\frac{1}{P_{\vec x}(x_{i})})$



In [1]:
def entropie(f_x):
   nz = np.where(f_x != 0)
   return -np.sum(f_x[nz] * np.log2(f_x[nz]))
print("Entropie H_x = ", entropie(f_1_x), "bit")

entropie
Entropie H_x =  12.89180477512903 bit

WDF für $d[n]$



In [1]:
d_L = np.diff(x_L[:,0])
# d_L = np.diff(d_L)
# d_L = np.diff(d_L)

f_1_d, bin_edge = np.histogram(d_L, bins_x, density=True)
plt.plot(bins_x[1:bins_x.size], f_1_d*100)
plt.xlabel("$x$")
plt.ylabel("rel. Haeufigkeit in %")

Text(0, 0.5, 'rel. Haeufigkeit in %')
[[./.ob-jupyter/804f492864b1c0ef0f212bdf0c45238759842e4e.png]]

Entropie des Differenzsignals



In [1]:
print("Entropie H_d = ", entropie(f_1_d), "bit")

Entropie H_d =  11.436124159935002 bit

Die Ersparnis durch die Differenzcodierung beträgt etwa



In [1]:
print(entropie(f_1_x)-entropie(f_1_d), " bit")

1.4556806151940282  bit

## Differenzen-Quantisierung



Nachfolgend finden Sie die Angaben aus der Exceltabelle des Abschnitts.  Vollziehen Sie die Quantisierung und Synthese des Ursprungssignals nach!

Die Grenzwerte der ungleichförmigen (und linearen) Quantisierung (nach der Tabelle  [https://vfhcmd.eduloop.de/loop/Differenzen:_Quantisierung](https://vfhcmd.eduloop.de/loop/Differenzen:_Quantisierung)) sind in der Abbildung dargestellt, die



In [1]:
Q_Gr = np.array([-10000, -26.5, -11.5, -4.5, -1.5, -0.5])
Q_Gr = np.concatenate((Q_Gr,  -Q_Gr[Q_Gr.size::-1]))
Q_Er = np.array([-38, -19, -8,  -3, -1, 0])
Q_Er = np.concatenate((Q_Er, -np.flip(Q_Er[:-1], axis=0)))

In [1]:
fig, ax = plt.subplots(1, 1, figsize=(10,2))
plt.stem(Q_Gr[1:Q_Gr.size-1], Q_Gr[1:Q_Gr.size-1], linefmt='b')
plt.stem(Q_Er[0:Q_Er.size], Q_Er[0:Q_Er.size], linefmt='r')
plt.xlabel("Eingangswerte x")
plt.ylabel("quantisierte Werte x_q")

Text(0, 0.5, 'quantisierte Werte x_q')
[[./.ob-jupyter/1e34b3b6645debb00743d74792ff727866d9da3a.png]]

### Quantisierer



Definiere Quantisierer



In [1]:
def quantisierer(x_lin_N, Q_Gr, Q_Er):
    x_qun_N = np.zeros(x_lin_N.size)

    for g in range(Q_Er.size): # suche für jeden Ersatzwert Kandidaten
        idx = np.logical_and(x_lin_N >  Q_Gr[g], x_lin_N <= Q_Gr[g+1]) # untere Grenze # obere Grenze
        x_qun_N[idx] = Q_Er[g] # weise Ersatzwert zu
    return x_qun_N

### Kennlinie



Die Darstellung der Kennlinie über linear aufsteigende Eingangswerte ist in der Abbildung dargestellt.



In [1]:
N = 100
x_lin_N = np.linspace(-50, 50, N)
plt.plot(x_lin_N, quantisierer(x_lin_N, Q_Gr, Q_Er))
plt.grid(True)
plt.xlabel("Eingangswerte $x$")
plt.ylabel("Ausgangswerte $x_{q}$")
# plt.savefig(filename)
# plt.close()
# Figure(filename, name=name, caption=caption, attributes=[('org', attr_org), ('latex', attr_latex)])

Text(0, 0.5, 'Ausgangswerte $x_{q}$')
[[./.ob-jupyter/b9382fe08db095301b97a618089406e5d01aa082.png]]

### quantisiertes Differenzsignal



### Quantisierungsfehler des Differenzsignals



### Synthese



synth. Signal aus d<sub>qn</sub><sub>N</sub> und d<sub>N</sub>



In [1]:
X_qun_N = dpcm_dec(d_qun_N)
X_N = dpcm_dec(d_N)

Die Abbildung zeigt die Darstellung der beiden Varianten und den kumulierten Fehler:



In [1]:
fig, ax = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]})
ax[0].plot(X_N, label="$x[n]$")
ax[0].plot(X_qun_N, label="$x_{q}[n]$")
ax[0].legend()
ax[0].grid(True)
ax[1].plot(e_ku_N[0:-1], label="$e_{kum}[n]$")
ax[1].set_xlabel("$n$")
ax[1].legend()
ax[1].grid(True)
# plt.savefig(filename)
# plt.close()
# Figure(filename, name=name, caption=caption, attributes=[('org', attr_org), ('latex', attr_latex)])

[[./.ob-jupyter/0624983143d0b2c0b59be68932377ce2968e4bcb.png]]